# STS Benchmark Datasets

For more information, see the implementation of https://github.com/kawine/usif

In [1]:
%load_ext autotime
import numpy as np
import logging
import pandas as pd
import pathlib

from gensim.models import Word2Vec, FastText
from gensim.models.keyedvectors import KeyedVectors, FastTextKeyedVectors

from fse.models.average import Average
from fse.models.sif import SIF
from fse.models.inputs import IndexedSentence

from re import sub

from scipy.stats import pearsonr

logging.basicConfig(format='%(asctime)s : %(threadName)s : %(levelname)s : %(message)s',
                    level=logging.INFO
                   )

file_path = "data/stsbenchmark/sts-test.csv"

def normalize_text(sentence):
    return [sub("[^a-zA-Z]", "", w.lower()) for w in sentence.split()] 

def compute_similarities(task_length, model):
    sims = []
    for i, j in zip(range(task_length), range(task_length, 2*task_length)):
        sims.append(model.sv.similarity(i,j))
    return sims

sts_data = pd.read_csv(file_path, sep=",", error_bad_lines=True, names=["cpt","1","task","3","sim","sent_a","sent_b"])
sts_data.dropna(inplace=True)
sts_data.sent_a = sts_data.sent_a.apply(normalize_text)
sts_data.sent_b = sts_data.sent_b.apply(normalize_text)
sts_data.task = sts_data.task.str.replace("2012test", "2012")
sts_data.task = sts_data.task.str.replace("2012train", "None")
sts_data = sts_data[sts_data.task != "None"]

w2v = KeyedVectors.load("/Users/oliverborchers/Desktop/GSDEV/Models/Static/google_news.model")
ft = FastTextKeyedVectors.load("/Users/oliverborchers/Desktop/GSDEV/Models/Static/ft_crawl_300d_2m.model")

tasks = sorted(["2012", "2013", "2014", "2015", "2016", "2017"])

2019-08-19 18:21:47,341 : MainThread : INFO : loading Word2VecKeyedVectors object from /Users/oliverborchers/Desktop/GSDEV/Models/Static/google_news.model
2019-08-19 18:21:53,096 : MainThread : INFO : loading vectors from /Users/oliverborchers/Desktop/GSDEV/Models/Static/google_news.model.vectors.npy with mmap=None
2019-08-19 18:21:57,559 : MainThread : INFO : setting ignored attribute vectors_norm to None
2019-08-19 18:21:57,560 : MainThread : INFO : loaded /Users/oliverborchers/Desktop/GSDEV/Models/Static/google_news.model
2019-08-19 18:21:57,561 : MainThread : INFO : loading FastTextKeyedVectors object from /Users/oliverborchers/Desktop/GSDEV/Models/Static/ft_crawl_300d_2m.model
2019-08-19 18:22:03,575 : MainThread : INFO : loading vectors from /Users/oliverborchers/Desktop/GSDEV/Models/Static/ft_crawl_300d_2m.model.vectors.npy with mmap=None
2019-08-19 18:22:06,281 : MainThread : INFO : loading vectors_vocab from /Users/oliverborchers/Desktop/GSDEV/Models/Static/ft_crawl_300d_2m.mo

In [2]:
w2v_avg = Average(w2v, wv_mapfile_path = "data/vectors/w2v", lang_freq="en")
ft_avg = Average(ft, wv_mapfile_path = "data/vectors/ft", lang_freq="en")

/Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/models/base_s2v.py:156: UserWarning: C extension not loaded, training/inferring will be slow. Install a C compiler and reinstall fse.
  "C extension not loaded, training/inferring will be slow. "
2019-08-19 18:22:13,539 : MainThread : INFO : loading pre-existing wv from data/vectors/w2v
2019-08-19 18:22:13,890 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en
2019-08-19 18:22:18,615 : MainThread : INFO : loading pre-existing wv from data/vectors/ft
2019-08-19 18:22:19,035 : MainThread : INFO : loading pre-existing ngrams from data/vectors/ft
2019-08-19 18:22:19,427 : MainThread : INFO : writing vocab to data/vectors/ft
2019-08-19 18:22:25,126 : MainThread : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en


time: 14 s


In [6]:
models, results = {}, {}

for year in tasks:
    models[f"CBOW-W2V-{year}"] = Average(w2v, wv_mapfile_path = "data/vectors/w2v")
    models[f"SIF-W2V-{year}"] = SIF(w2v, wv_mapfile_path = "data/vectors/w2v")
    
    models[f"CBOW-FT-{year}"] = Average(ft, wv_mapfile_path = "data/vectors/ft")
    models[f"SIF-FT-{year}"] = SIF(ft, wv_mapfile_path = "data/vectors/ft")
    
for k in models:
    results[k] = 0

/Users/oliverborchers/Library/Mobile Documents/com~apple~CloudDocs/Diss/Medium/Fast_Sentence_Embeddings/fse/models/base_s2v.py:156: UserWarning: C extension not loaded, training/inferring will be slow. Install a C compiler and reinstall fse.
  "C extension not loaded, training/inferring will be slow. "
2019-08-19 18:25:01,749 : MainThread : INFO : loading pre-existing wv from data/vectors/w2v
2019-08-19 18:25:01,767 : MainThread : INFO : loading pre-existing wv from data/vectors/w2v
2019-08-19 18:25:01,787 : MainThread : INFO : loading pre-existing wv from data/vectors/ft
2019-08-19 18:25:01,803 : MainThread : INFO : loading pre-existing ngrams from data/vectors/ft
2019-08-19 18:25:01,821 : MainThread : INFO : loading pre-existing vocab from data/vectors/ft
2019-08-19 18:25:01,840 : MainThread : INFO : loading pre-existing wv from data/vectors/ft
2019-08-19 18:25:01,854 : MainThread : INFO : loading pre-existing ngrams from data/vectors/ft
2019-08-19 18:25:01,871 : MainThread : INFO : 

time: 840 ms


In [7]:
for year in tasks:
    df = sts_data[sts_data.task == year]
    sents_a = df.sent_a.values.tolist()
    sents_b = df.sent_b.values.tolist()
    assert len(sents_a) == len(sents_b)

    task_length = len(sents_a)
    sents_a.extend(sents_b)
    
    sents = [IndexedSentence(s, i) for i,s in enumerate(sents_a)]
    
    for k, m in models.items():
        if k.split("-")[-1] != year:
            continue
        
        m_type  = k.split("-")[0]
        emb_type = k.split("-")[1]
        
        m.train(sents)
        r = pearsonr(df.sim, compute_similarities(task_length, m))[0].round(4)
        
        results[f"{m_type}-{emb_type}-{year}"] = r * 100
        print(k, r)

2019-08-19 18:25:03,726 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-19 18:25:03,730 : MainThread : INFO : finished scanning 504 sentences with an average length of 13 and 6848 total words
2019-08-19 18:25:04,920 : MainThread : INFO : estimated memory for 504 sentences with 300 dimensions and 3000000 vocabulary: 3445 MB (3 GB)
2019-08-19 18:25:04,921 : MainThread : INFO : initializing sentence vectors for 504 sentences
2019-08-19 18:25:04,985 : MainThread : INFO : begin training
2019-08-19 18:25:05,035 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-19 18:25:05,036 : MainThread : INFO : training on 504 effective sentences with 5060 effective words took 0s with 9805 sentences/s
2019-08-19 18:25:05,046 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-19 18:25:05,052 : MainThread : INFO : finished scanning 504 sentences with an average length of 13 and 6848 total words


CBOW-W2V-2012 0.7467


2019-08-19 18:25:06,426 : MainThread : INFO : estimated memory for 504 sentences with 300 dimensions and 3000000 vocabulary: 3445 MB (3 GB)
2019-08-19 18:25:06,427 : MainThread : INFO : initializing sentence vectors for 504 sentences
2019-08-19 18:25:06,431 : MainThread : INFO : pre-computing SIF weights for 3000000 words
2019-08-19 18:25:11,856 : MainThread : INFO : begin training
2019-08-19 18:25:11,904 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-19 18:25:11,905 : MainThread : INFO : computing 1 principal components
2019-08-19 18:25:11,913 : MainThread : INFO : removing 1 principal components
2019-08-19 18:25:11,915 : MainThread : INFO : training on 504 effective sentences with 5060 effective words took 0s with 10203 sentences/s
2019-08-19 18:25:11,930 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-19 18:25:11,935 : MainThread : INFO : finished scanning 504 sentences with an average length of 13 and 6848 

SIF-W2V-2012 0.7874


2019-08-19 18:25:12,895 : MainThread : WARNING : Fasttext memory estimation not yet completed
2019-08-19 18:25:12,896 : MainThread : INFO : estimated memory for 504 sentences with 300 dimensions and 2000000 vocabulary: 2297 MB (2 GB)
2019-08-19 18:25:12,897 : MainThread : INFO : initializing sentence vectors for 504 sentences
2019-08-19 18:25:12,941 : MainThread : INFO : begin training
2019-08-19 18:25:13,050 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-19 18:25:13,051 : MainThread : INFO : training on 504 effective sentences with 6848 effective words took 0s with 4607 sentences/s
2019-08-19 18:25:13,060 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-19 18:25:13,063 : MainThread : INFO : finished scanning 504 sentences with an average length of 13 and 6848 total words


...

SIF-W2V-2017 0.7708


2019-08-19 18:26:09,012 : MainThread : WARNING : Fasttext memory estimation not yet completed
2019-08-19 18:26:09,012 : MainThread : INFO : estimated memory for 250 sentences with 300 dimensions and 2000000 vocabulary: 2296 MB (2 GB)
2019-08-19 18:26:09,013 : MainThread : INFO : initializing sentence vectors for 250 sentences
2019-08-19 18:26:09,046 : MainThread : INFO : begin training
2019-08-19 18:26:09,073 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-19 18:26:09,073 : MainThread : INFO : training on 250 effective sentences with 2319 effective words took 0s with 9339 sentences/s
2019-08-19 18:26:09,077 : MainThread : INFO : scanning all indexed sentences and their word counts
2019-08-19 18:26:09,078 : MainThread : INFO : finished scanning 250 sentences with an average length of 9 and 2319 total words


CBOW-FT-2017 0.5091


2019-08-19 18:26:09,869 : MainThread : WARNING : Fasttext memory estimation not yet completed
2019-08-19 18:26:09,870 : MainThread : INFO : estimated memory for 250 sentences with 300 dimensions and 2000000 vocabulary: 2296 MB (2 GB)
2019-08-19 18:26:09,870 : MainThread : INFO : initializing sentence vectors for 250 sentences
2019-08-19 18:26:09,872 : MainThread : INFO : pre-computing SIF weights for 2000000 words
2019-08-19 18:26:12,149 : MainThread : INFO : begin training
2019-08-19 18:26:12,175 : MainThread : INFO : worker thread finished; awaiting finish of 0 more threads
2019-08-19 18:26:12,176 : MainThread : INFO : computing 1 principal components
2019-08-19 18:26:12,179 : MainThread : INFO : removing 1 principal components
2019-08-19 18:26:12,180 : MainThread : INFO : training on 250 effective sentences with 2319 effective words took 0s with 9215 sentences/s


SIF-FT-2017 0.7487
time: 1min 8s


In [8]:
pd.DataFrame.from_dict(results, orient="index", columns=["Pearson"])

,Pearson
CBOW-W2V-2012,74.67
SIF-W2V-2012,78.74
CBOW-FT-2012,50.99
SIF-FT-2012,79.10
CBOW-W2V-2013,64.74
SIF-W2V-2013,67.41
CBOW-FT-2013,65.06
SIF-FT-2013,70.39
CBOW-W2V-2014,71.05
SIF-W2V-2014,75.50


time: 7.5 ms
